# Лабораторная работа №3
## Выполнил студент группы БСТ2003 Григорян Арсен Григорович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
from datetime import datetime as t
import collections
from queue import PriorityQueue

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [4]:
def kmp(stroka,podstroka):
    p = [0]*len(podstroka) # массив pi
    j = 0
    i = 1
    
    # формирование массива p
    while i < len(podstroka):
        if podstroka[j] == podstroka[i]:
            p[i] = j+1
            i += 1
            j += 1
        else:
            if j == 0:
                p[i] = 0;
                i += 1
            else:
                j = p[j-1]

    print(p)

    m = len(podstroka)
    n = len(stroka)

    i = 0
    j = 0
    while i < n:
        if stroka[i] == podstroka[j]:
            i += 1
            j += 1
            if j == m:
                print(f"образ найден по индексу {i-(len(podstroka))}")
                break
        else:
            if j > 0:
                j = p[j-1]
            else:
                i += 1

    if i == n and j != m:
        print("образ не найден")
        
stroka = str(input("Введите строку: "))
podstroka = str(input("Введите подстроку: "))
ans = str(input("Чувствительность к регистру Да/Нет: ").lower())
if ans == "нет":
    start_time = t.now()
    kmp(stroka.lower(),podstroka.lower())
    print(t.now() - start_time)
elif ans == "да": 
    start_time = t.now()
    kmp(stroka,podstroka)
    print(t.now() - start_time)
else: print("Введите корректный ответ")

Введите строку: abcabeabcabcabd
Введите подстроку: abcabd
Чувствительность к регистру Да/Нет: Нет
[0, 0, 0, 1, 2, 0]
образ найден по индексу 9
0:00:00


#### Упрощенный алгоритм Бойера-Мура

In [5]:
def BM(stroka,podstroka):

    S = set()  
    M = len(podstroka) 
    d = {}     

    for i in range(M-2, -1, -1): 
        if podstroka[i] not in S:        
            d[podstroka[i]] = M-i-1
            S.add(podstroka[i])

    if podstroka[M-1] not in S:     
        d[podstroka[M-1]] = M

    d['*'] = M            

    print(d)

    N = len(stroka)

    if N >= M:
        i = M-1     

        while(i < N):
            k = 0
            j = 0
            flBreak = False
            for j in range(M-1, -1, -1):
                if stroka[i-k] != podstroka[j]:
                    if j == M-1:
                        off = d[stroka[i]] if d.get(stroka[i], False) else d['*']  # смещение, если не равен последний символ образа
                    else:
                        off = d[podstroka[j]]   

                    i += off    
                    flBreak = True  
                    break

                k += 1          

            if not flBreak:          
                print(f"образ найден по индексу: {i-(M-1)}")
                break
        else:
            print("образ не найден")
    else:
        print("образ не найден")
        
stroka = str(input("Введите строку: "))
podstroka = str(input("Введите подстроку: "))
ans = str(input("Чувствительность к регистру Да/Нет: ").lower())
if ans == "нет":
    start_time = t.now()
    BM(stroka.lower(),podstroka.lower())
    print(t.now() - start_time)
elif ans == "да": 
    start_time = t.now()
    BM(stroka,podstroka)
    print(t.now() - start_time)
else: print("Введите корректный ответ") 

Введите строку: выиграть матч
Введите подстроку: матч
Чувствительность к регистру Да/Нет: Нет
{'т': 1, 'а': 2, 'м': 3, 'ч': 4, '*': 4}
образ найден по индексу: 9
0:00:00.000081


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [6]:
revprint = collections.deque()
class Position:
    def __init__(self, position, start_distance,finish_distance):
        self.position = position
        self.start_distance = start_distance
        self.finish_distance = finish_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

    def __lt__(self, other):
        return self.start_distance+self.finish_distance < other.start_distance+other.finish_distance
N = 4
def shifts(position):
    zeroPosition = position.index(0)
    i, j = divmod(zeroPosition, N)
    displacement = []
    if i > 0: displacement.append(-N)     
    if i < N - 1: displacement.append(N)  
    if j > 0: displacement.append(-1)     
    if j < N - 1: displacement.append(1)  
    for offset in displacement:
        swap = zeroPosition + offset
        yield tuple(position[swap]
        if x==zeroPosition
        else position[zeroPosition]
        if x==swap
        else position[x]
        for x in range(N*N))


def parityOfPairs(state):
    countOfPairs = 0
    for i in range(len(state)-1):
        if state[i] > state[i+1]:
            countOfPairs +=1
    return countOfPairs % 2
def fifteenGame(startState):
    terminalState = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
    if parityOfPairs(startState)==0:
        print("Нет решений")
    else:
        startState= tuple(startState)
        p = Position(startState, 0,0)

        fieldStates= PriorityQueue()
        fieldStates.put(p)
        closePoints = set([p])
        parents = {p.position: None}

        while p.position != terminalState:
            p =fieldStates.get()


            for k in shifts(p.position):
                count= 0
                if k not in closePoints:
                    for m in range(len(k)):
                        if k[m] != terminalState[m]:
                            count+=1


                    fieldStates.put(Position(k, p.start_distance +1,p.finish_distance+count))
                    parents[k] = p

                    closePoints.add(k)

        path = []
        x = p
        previous = p
        while p.position != startState:

            p = parents[p.position]

            number = p.position[previous.position.index(0)]
            path.append(number)
            previous = p
            revprint.append(p)

        path.reverse()
    if parityOfPairs(startState)!=0:
        for reverse_print in range(len(path)):
            print(f"\n{revprint.pop()}")
        print(f"\n{x}\n\n{path}\nЧисло шагов: {len(path)}")
startState = [1,2,3,4,
              5,6,7,8,
              9,10,11,12,
              0,13,14,15]
fifteenGame(startState)


  1  2  3  4
  5  6  7  8
  9 10 11 12
  0 13 14 15

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13  0 14 15

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14  0 15

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15  0

[13, 14, 15]
Число шагов: 3


### Вывод